In [1]:
test_input = """.|...\\....\n|.-.\\.....\n.....|-...\n........|.\n..........\n.........\\\n..../.\\\\..\n.-.-/..|..\n.|....-|.\\\n..//.|...."""
with open('day16.txt', 'rt') as f: raw_input  = f.read()

In [2]:
from collections import namedtuple
import copy
from queue import deque

Move = namedtuple('Move', ['dx', 'dy'])
Position = namedtuple('Position', ['x', 'y'])
Matrix = list[list[str]]

SPACE = '.'
MIRROR_LEFT = '/'
MIRROR_RIGHT = '\\'
SPLITTER_VERTICAL = '|'
SPLITTER_HORIZONTAL = '-'
ENERGIZED = '#'

UP = Move(dx=0, dy=-1)
DOWN = Move(dx=0, dy=1)
LEFT = Move(dx=-1, dy=0)
RIGHT = Move(dx=1, dy=0)
UP_RIGHT = Move(dx=1, dy=-1)
DOWN_RIGHT = Move(dx=1, dy=1)
UP_LEFT = Move(dx=-1, dy=-1)
DOWN_LEFT = Move(dx=-1, dy=1)

BEHAVIORS: dict[tuple[Move, str], list[Move]] = {
    (RIGHT, MIRROR_LEFT): [UP],
    (LEFT, MIRROR_LEFT): [DOWN],
    (UP, MIRROR_LEFT): [RIGHT],
    (DOWN, MIRROR_LEFT): [LEFT],
    (RIGHT, MIRROR_RIGHT): [DOWN],
    (LEFT, MIRROR_RIGHT): [UP],
    (DOWN, MIRROR_RIGHT): [RIGHT],
    (UP, MIRROR_RIGHT): [LEFT],
    (RIGHT, SPLITTER_VERTICAL): [UP, DOWN],
    (LEFT, SPLITTER_VERTICAL): [UP, DOWN],
    (UP, SPLITTER_HORIZONTAL): [LEFT, RIGHT],
    (DOWN, SPLITTER_HORIZONTAL): [LEFT, RIGHT],
}


def is_valid(matrix: Matrix, pos: Position) -> bool:
    return pos.x >= 0 and pos.y >= 0 and pos.x < len(matrix[0]) and pos.y < len(matrix)


def get_item(matrix: Matrix, pos: Position) -> str:
    return matrix[pos.y][pos.x]


def assign_item(matrix: Matrix, pos: Position, value: str):
#     if get_item(matrix, pos) == SPACE:
#         matrix[pos.y][pos.x] = value
    matrix[pos.y][pos.x] = value


def print_matrix(matrix: Matrix):
    for row in matrix:
        print(''.join(row))
    print('________________________________')


def calculate_energized(matrix: Matrix, start_pos: Position, start_move: Move):
    energized_matrix = copy.deepcopy(matrix)
    stacks = deque()
    stacks.append((start_pos, start_move))
    closed_stacks = set()

    while stacks:
        current_pos, move = stacks.popleft()
        if (current_pos, move) in closed_stacks:
            continue
        else:
            closed_stacks.add((current_pos, move))
        next_pos = Position(x=current_pos.x + move.dx, y=current_pos.y + move.dy)
        if not is_valid(matrix=matrix, pos=next_pos):
            continue
        next_pos_value = get_item(matrix=matrix, pos=next_pos)
#         print(current_pos, move, next_pos_value, BEHAVIORS.get((move, next_pos_value)))
#         print_matrix(energized_matrix)
        assign_item(matrix=energized_matrix, pos=next_pos, value=ENERGIZED)
        behavior_moves = BEHAVIORS.get((move, next_pos_value))
        
        if not behavior_moves:
            stacks.append((next_pos, move))
        else:
            for behavior_move in behavior_moves:
                behavior_pos = Position(x=next_pos.x + behavior_move.dx, y=next_pos.y + behavior_move.dy)
                if is_valid(matrix=matrix, pos=behavior_pos):
                    stacks.append((next_pos, behavior_move))
    
    return energized_matrix


def count_energized(matrix: Matrix):
    return sum([row.count(ENERGIZED) for row in matrix])


def solve1(raw_input: str):
    matrix = list(map(list, raw_input.strip().splitlines()))
    energized_matrix = calculate_energized(matrix, start_pos=Position(x=-1, y=0), start_move=RIGHT)
#     print_matrix(energized_matrix)
#     print([row.count(ENERGIZED) for row in energized_matrix])
    return count_energized(energized_matrix)

def solve2(raw_input: str):
    matrix = list(map(list, raw_input.strip().splitlines()))
    max_energized, max_energized_matrix = 0, None
    start_ways = []
    start_ways.extend([(Position(x=x, y=-1), DOWN) for x in range(len(matrix[0]))])
    start_ways.extend([(Position(x=x, y=len(matrix)), UP) for x in range(len(matrix[0]))])
    start_ways.extend([(Position(x=-1, y=y), RIGHT) for y in range(len(matrix))])
    start_ways.extend([(Position(x=len(matrix[0]), y=y), LEFT) for y in range(len(matrix))])
    for start_pos, start_move in start_ways:
        energized_matrix = calculate_energized(matrix, start_pos, start_move)
        energized = count_energized(energized_matrix)
        if energized > max_energized:
            max_energized = energized
            max_energized_matrix = energized_matrix
#     print_matrix(max_energized_matrix)
    return max_energized

In [3]:
solve1(raw_input)

6605

In [4]:
solve2(raw_input)

6766